<a href="https://colab.research.google.com/github/varshacvenkat-web/Varsha-Venkatapathy-Engineering-Portfolio-/blob/main/Preprocessing_Article_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import librosa
import numpy as np
from scipy.signal import get_window
from librosa.filters import mel

# Paths
CLEAN_SPEECH_PATH = "path_to_clean_speech"
NOISE_PATH = "path_to_noise"
OUTPUT_PATH = "path_to_output"

# Parameters
SAMPLE_RATE = 8000
FRAME_SIZE = 256  # 32 ms at 8 kHz
HOP_LENGTH = 128  # 50% overlap
NUM_MEL_BINS = 26

def load_audio(file_path, sample_rate=SAMPLE_RATE):
    """Load audio file and resample."""
    audio, sr = librosa.load(file_path, sr=sample_rate)
    return audio

def apply_fft(audio):
    """Apply FFT to audio frames."""
    window = get_window("hann", FRAME_SIZE, fftbins=True)
    stft_matrix = librosa.stft(audio, n_fft=FRAME_SIZE, hop_length=HOP_LENGTH, window=window)
    magnitude = np.abs(stft_matrix)
    return magnitude

def compute_lps(magnitude):
    """Compute Log Power Spectrum."""
    power_spectrum = magnitude**2
    lps = np.log(np.maximum(power_spectrum, 1e-8))  # Avoid log(0)
    return lps

def compute_mfsc(magnitude, sample_rate=SAMPLE_RATE, n_mel_bins=NUM_MEL_BINS):
    """Compute Log Mel-Filterbank Coefficients."""
    mel_filters = mel(sample_rate, n_fft=FRAME_SIZE, n_mels=n_mel_bins, fmin=300, fmax=4000)
    mel_energy = np.dot(mel_filters, magnitude**2)
    log_mel_energy = np.log(np.maximum(mel_energy, 1e-8))  # Avoid log(0)
    return log_mel_energy

def add_noise(clean_audio, noise_audio, snr_db):
    """Add noise to clean audio at a specific SNR."""
    clean_rms = np.sqrt(np.mean(clean_audio**2))
    noise_rms = np.sqrt(np.mean(noise_audio**2))
    snr_linear = 10**(snr_db / 20)
    scaled_noise = noise_audio * (clean_rms / (snr_linear * noise_rms))
    noisy_audio = clean_audio + scaled_noise
    return noisy_audio

def process_audio(clean_path, noise_path, output_path, snr_levels=[-5, 0, 5]):
    """Main processing pipeline."""
    os.makedirs(output_path, exist_ok=True)
    clean_audio = load_audio(clean_path)
    noise_audio = load_audio(noise_path)

    for snr in snr_levels:
        # Add noise to clean speech
        noisy_audio = add_noise(clean_audio, noise_audio, snr)

        # Frame the audio and compute FFT
        clean_magnitude = apply_fft(clean_audio)
        noisy_magnitude = apply_fft(noisy_audio)

        # Compute LPS and MFSC
        clean_lps = compute_lps(clean_magnitude)
        noisy_lps = compute_lps(noisy_magnitude)

        clean_mfsc = compute_mfsc(clean_magnitude)
        noisy_mfsc = compute_mfsc(noisy_magnitude)

        # Concatenate features for temporal context
        clean_features = np.concatenate([clean_lps, clean_mfsc], axis=0)
        noisy_features = np.concatenate([noisy_lps, noisy_mfsc], axis=0)

        # Save the features
        np.save(os.path.join(output_path, f"clean_features_snr{snr}.npy"), clean_features)
        np.save(os.path.join(output_path, f"noisy_features_snr{snr}.npy"), noisy_features)

# Example usage
clean_files = os.listdir(CLEAN_SPEECH_PATH)
noise_files = os.listdir(NOISE_PATH)

for clean_file in clean_files:
    for noise_file in noise_files:
        process_audio(
            os.path.join(CLEAN_SPEECH_PATH, clean_file),
            os.path.join(NOISE_PATH, noise_file),
            OUTPUT_PATH,
            snr_levels=[-5, 0, 5]
        )
